<a href="https://colab.research.google.com/github/eliasab16/MLectric/blob/main/image_classifier_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

%cd drive/Othercomputers/My\ MacBook\ Pro/tf-od/panels/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Othercomputers/My MacBook Pro/tf-od/panels


In [ ]:
!pip install tensorflowjs

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, BatchNormalization, Dropout
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow import keras
import pandas as pd
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import random
import json
import pickle

In [ ]:
CLASSIFIER_PATH = os.path.join('workspace/classifier')

paths = {
    'TRAINED_MODELS': os.path.join(CLASSIFIER_PATH, 'trained_models')
}

## Data prep and augmentation

### Convert data into table

In [ ]:
label_names = {
    0: 'af', 1: 'bhaat2x40', 2: 'bhaat4x40', 3: 'breaker3x', 4: 'm1x6', 5: 'm1x10', 6: 'm1x16',
    7: 'm1x20', 8: 'm1x25', 9: 'm1x32', 10: 'm1x40', 11: 'm3x6', 12: 'm3x10', 13: 'm3x16',
    14: 'm3x20', 15: 'm3x25', 16: 'm3x32', 17: 'm3x40', 18: 'm3x50', 19: 'm3x63',
    20: 'ms'
}

label_nums = {
    'af': 0, 'bhaat2x40': 1, 'bhaat4x40': 2, 'breaker3x': 3, 'm1x6': 4, 'm1x10': 5, 'm1x16': 6,
    'm1x20': 7, 'm1x25': 8, 'm1x32': 9, 'm1x40': 10, 'm3x6': 11, 'm3x10': 12, 'm3x16': 13,
    'm3x20': 14, 'm3x25': 15, 'm3x32': 16, 'm3x40': 17, 'm3x50': 18, 'm3x63': 19,
    'ms': 20
}

In [ ]:
# Combine images, categories, and labels in a table

data_table = {'image_path': [], 'label': []}
root_folder = "workspace/classifier/data"

for label_name in os.listdir(root_folder):
    label = label_nums[label_name]
    folder_path = os.path.join(root_folder, label_name)

    for image_file in os.listdir(folder_path):
        if image_file.lower().endswith('.jpg'):
            image_path = os.path.join(folder_path, image_file)
            data_table['image_path'].append(image_path)
            data_table['label'].append(label)

# Create a Pandas DataFrame from the data dictionary
original_df = pd.DataFrame(data_table)

# Print the DataFrame
print(original_df)


### Agumentation #2

#### Augmentation

In [ ]:
augmentation_generator = ImageDataGenerator(
    rotation_range=0.8,
    height_shift_range=0.08,
    width_shift_range=0.12,
    rescale=1.0/255.0
)

In [ ]:
augmentation map = []

In [ ]:
augmented_data = []
root_folder = "workspace/classifier/data"

for (target_label, iterations) in augmentation_map:
  filtered_table = original_df[original_df['label'] == target_label]

  for _ in range(iterations):
    random_row = filtered_table.sample(n=1)
    img_path = random_row['image_path'].values[0]
    label = random_row['label'].values[0]
    img = load_img(img_path, target_size=(256, 256))
    img_array = img_to_array(img)

    augmented_images = augmentation_generator.random_transform(img_array)
    augmented_data.append({
        'image': augmented_images,
        'label': label
    })

  for index, row in filtered_table.iterrows():
    label = row['label']
    img_path = row['image_path']
    img = load_img(img_path, target_size=(256, 256))
    img_array = img_to_array(img)

    augmented_data.append({
        'image': img_array,
        'label': label
    })

  # Convert the augmented data to a new dataframe
  augmented_df = pd.DataFrame(augmented_data)

#### Downsample

In [ ]:
# drop rows to downsample
dataframe = augmented_df

for label in dataframe['label'].unique():
  target_label = label

  num_rows = len(dataframe[dataframe['label'] == target_label]) - 190

  if num_rows > 0:
    # Get the indices of rows with the target label
    rows_to_drop = dataframe[dataframe['label'] == target_label].sample(num_rows).index

    # Drop the specified rows
    dataframe = dataframe.drop(rows_to_drop)

augmented_df = dataframe

In [ ]:
augmented_df['label'].value_counts()

#### Save data

In [ ]:
for _, row in augmented_df.iterrows():
  image_np = row['image']
  # Convert the numpy array to an image
  image = Image.fromarray((np.array(image_np)).astype(np.uint8))

  # Display the image using matplotlib
  plt.figure()
  plt.imshow(image)
  plt.axis('off')  # Turn off axis
  plt.show()

In [ ]:
# Save the dataframe
%cd workspace/classifier
augmented_df.to_csv('augmented_df.csv', index=False)
%cd ../..

In [ ]:
# Load df
%cd workspace/classifier
df = pd.read_csv('augmented_df.csv')
%cd ../..

In [ ]:
# Save only augmented images
stacked_augmented_df = np.stack(np.array(augmented_df['image']))
%cd workspace/classifier
np.save('augmented_array.npy', stacked_augmented_df)
%cd ../..

/content/drive/Othercomputers/My MacBook Pro/tf-od/panels/workspace/classifier
/content/drive/Othercomputers/My MacBook Pro/tf-od/panels


In [ ]:
# Load augmented images array
%cd workspace/classifier
loaded_array = np.load('augmented_array.npy')
%cd ../..

/content/drive/Othercomputers/My MacBook Pro/tf-od/panels/workspace/classifier
/content/drive/Othercomputers/My MacBook Pro/tf-od/panels


### Save as csv

In [ ]:
import ast

# Function to convert Numpy arrays to strings
def np_array_to_string(array):
    return ','.join(map(str, array))

# Function to convert strings to Numpy arrays
def string_to_np_array(string):
    return np.array(ast.literal_eval(string))

In [ ]:
expdf = pd.DataFrame(columns=augmented_df.columns)

In [ ]:
expdf = augmented_df

In [ ]:
%cd workspace/classifier

'/content/drive/Othercomputers/My MacBook Pro/tf-od/panels'

In [ ]:
# Serialize the NumPy arrays using pickle
expdf['image'] = expdf['image'].apply(lambda arr: pickle.dumps(arr))

# Save the DataFrame to a CSV file
expdf.to_csv('data.csv', index=False)

# Load the CSV file back into a DataFrame
loaded_df = pd.read_csv('data.csv')

# Deserialize the pickled arrays back to NumPy arrays
loaded_df['image'] = loaded_df['image'].apply(lambda arr_pickle: pickle.loads(eval(arr_pickle)))

# Print the original and loaded arrays for verification
print("Original DataFrame:\n", expdf)
print("\nLoaded DataFrame:\n", loaded_df)


### Split into training and testing dataframes

In [ ]:
def split_data(data_df, train_fraction=0.8):
  if train_fraction <= 0 or train_fraction > 1:
    raise ValueError("train_fraction must be larger than 0 and less than or equal 1")

  # Shuffle the data
  unique_labels = data_df['label'].unique()

  training = pd.DataFrame(columns=data_df.columns)
  testing = pd.DataFrame(columns=data_df.columns)

  for label in unique_labels:
      label_data = data_df[data_df['label'] == label]

      num_testing_rows = max(int(len(label_data) * (1-train_fraction)), 1)

      testing_indices = np.random.choice(label_data.index, size=num_testing_rows, replace=False)

      testing = pd.concat([testing, label_data.loc[testing_indices]])

  # Rows not in testing_df are automatically in training df
  training = data_df.drop(testing.index)

  print("Training dataframe size:", len(training))
  print("Testing dataframe size:", len(testing))

  return training, testing


#### Save splits

In [ ]:
# TRAINING
# Save only augmented images
training_augmented_images = np.stack(np.array(training_df['image']))
%cd workspace/classifier/training
np.save('images_array.npy', training_augmented_images)
%cd ../../..

/content/drive/Othercomputers/My MacBook Pro/tf-od/panels/workspace/classifier/training
/content/drive/Othercomputers/My MacBook Pro/tf-od/panels/workspace


In [ ]:
# TRAINING
# Save the dataframe
%cd workspace/classifier/training
training_df[['category', 'label']].to_csv('training_df.csv', index=False)
%cd ../../..

/content/drive/Othercomputers/My MacBook Pro/tf-od/panels/workspace/classifier/training
/content/drive/Othercomputers/My MacBook Pro/tf-od/panels


In [ ]:
# TESTING
# Save only augmented images
testing_augmented_images = np.stack(np.array(testing_df['image']))
%cd workspace/classifier/testing
np.save('images_array.npy', testing_augmented_images)
%cd ../../..

/content/drive/Othercomputers/My MacBook Pro/tf-od/panels/workspace/classifier/testing
/content/drive/Othercomputers/My MacBook Pro/tf-od/panels


In [ ]:
# TESTING
# Save the dataframe
%cd workspace/classifier/testing
testing_df[['category', 'label']].to_csv('testing_df.csv', index=False)
%cd ../../..

/content/drive/Othercomputers/My MacBook Pro/tf-od/panels/workspace/classifier/testing
/content/drive/Othercomputers/My MacBook Pro/tf-od/panels


## Building and training the model

### Model definitions

#### Custom model

In [ ]:
num_classes=len(training_df['label'].unique())

# Image
image_input = Input(shape=(256, 256, 3), name='image_input')
conv1 = Conv2D(16, (3, 3), 1, activation='relu', kernel_regularizer=l2(0.3))(image_input)
bn1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D()(bn1)
drop1 = Dropout(0.4)(pool1)

conv2 = Conv2D(32, (3, 3), 1, activation='relu', kernel_regularizer=l2(0.3))(drop1)
bn2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D()(bn2)
drop2 = Dropout(0.4)(pool2)

conv3 = Conv2D(16, (3, 3), 1, activation='relu', kernel_regularizer=l2(0.3))(drop2)
bn3 = BatchNormalization()(conv3)
pool3 = MaxPooling2D()(bn3)
drop3 = Dropout(0.4)(pool3)

flat = Flatten()(drop3)
dense1 = Dense(256, activation='relu')(flat)

# category input
category_input = Input(shape=(1,), name='category_input')
dense2 = Dense(64)(category_input)

# combine image and category features
combined = Concatenate()([dense1, dense2])

# output layer for multi-class classification
output = Dense(num_classes, activation='softmax')(combined)

model = Model(inputs=[image_input, category_input], outputs=output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### AlexNet

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.metrics import categorical_crossentropy
# from keras.optimizers import SGD
from tensorflow.keras.optimizers.legacy import SGD

In [ ]:
def AlexNet_regularization(num_classes, learning_rate=0.0005, momentum=0.9, decay=0, regularization=0.2):
  model = Sequential()

  # layer 1: convolutional layer + max-pooling layer
  model.add(Conv2D(filters = 96, kernel_size = (11,11), strides= 4, padding = 'valid', activation='relu', input_shape = (256, 256, 3), kernel_regularizer=l2(regularization)))
  model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

  # layer 2: convolutional layer + max-pooling layer
  model.add(Conv2D(filters = 256, kernel_size = (5,5), padding = 'same', activation = 'relu', kernel_regularizer=L1L2(l2=regularization)))
  model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

  # layers 3-5: three convolutional layers + 1 max-pooling layer
  model.add(Conv2D(filters = 384, kernel_size = (3,3), padding = 'same', activation = 'relu', kernel_regularizer=L1L2(l2=regularization)))
  model.add(Conv2D(filters = 384, kernel_size = (3,3), padding = 'same', activation = 'relu', kernel_regularizer=L1L2(l2=regularization)))
  model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = 'same', activation = 'relu', kernel_regularizer=L1L2(l2=regularization)))
  model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

  # layers 6 - 8: two fully connected hidden layers and one fully connected output layer
  model.add(Flatten())
  model.add(Dense(4096, activation = 'relu'))
  model.add(Dropout(0.5))
  model.add(Dense(4096, activation = 'relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation = 'softmax'))

  optimizer = SGD(lr=learning_rate, momentum=momentum, decay=decay)
  # optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.0001)
  model.compile(loss = categorical_crossentropy,
                optimizer = optimizer,
                metrics = ['accuracy'])

  return model

In [ ]:
def AlexNet(num_classes, learning_rate=1e-04, momentum=0.9, decay=0):
  model = Sequential()

  # layer 1: convolutional layer + max-pooling layer
  model.add(Conv2D(filters = 96, kernel_size = (11,11), strides= 4, padding = 'valid', activation='relu', input_shape = (256, 256, 3)))
  model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

  # layer 2: convolutional layer + max-pooling layer
  model.add(Conv2D(filters = 256, kernel_size = (5,5), padding = 'same', activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

  # layers 3-5: three convolutional layers + 1 max-pooling layer
  model.add(Conv2D(filters = 384, kernel_size = (3,3), padding = 'same', activation = 'relu'))
  model.add(Conv2D(filters = 384, kernel_size = (3,3), padding = 'same', activation = 'relu'))
  model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = 'same', activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

  # layers 6 - 8: two fully connected hidden layers and one fully connected output layer
  model.add(Flatten())
  model.add(Dense(4096, activation = 'relu'))
  model.add(Dropout(0.5))
  model.add(Dense(4096, activation = 'relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation = 'softmax'))

  optimizer = SGD(lr=learning_rate, momentum=momentum, decay=decay)
  # optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.0001)
  model.compile(loss = categorical_crossentropy,
                optimizer = optimizer,
                metrics = ['accuracy'])

  return model

#### AlexNet 2-inputs

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

def AlexNetMixedInput(num_classes):
    # Image input branch
    image_input = Input(shape=(256, 256, 3), name='image_input')
    x = Conv2D(filters=96, kernel_size=(11, 11), strides=4, padding='valid', activation='relu')(image_input)
    x = MaxPooling2D(pool_size=(3, 3), strides=2)(x)
    x = Conv2D(filters=256, kernel_size=(5, 5), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=2)(x)
    x = Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=2)(x)
    x = Flatten()(x)

    # category input branch
    category_input = Input(shape=(1,), name='category_input')
    y = Dense(64, activation='relu')(category_input)

    # concatenate image and category branches
    combined = tf.keras.layers.concatenate([x, y], axis=-1)

    # fully connected layers
    z = Dense(4096, activation='relu')(combined)
    z = Dropout(0.5)(z)
    z = Dense(4096, activation='relu')(z)
    z = Dropout(0.5)(z)
    output = Dense(num_classes, activation='softmax')(z)

    # create the model with two inputs and one output
    model = Model(inputs=[image_input, category_input], outputs=output)

    # optimizer = SGD(learning_rate=0.00001, momentum=0.6)
    optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    return model

### Train model

#### Load data

In [ ]:
# Load augmented images array
%cd workspace/classifier
loaded_array = np.load('augmented_array.npy')
loaded_df = pd.read_csv('augmented_df.csv')
%cd ../..

/content/drive/Othercomputers/My MacBook Pro/tf-od/panels/workspace/classifier
/content/drive/Othercomputers/My MacBook Pro/tf-od/panels


In [ ]:
filtered_df = loaded_df[['label']]

In [ ]:
split_arrays = [np.array(loaded_array[i]) for i in range(loaded_array.shape[0])]
filtered_df['image'] = split_arrays

In [ ]:
training_df, testing_df = split_data(filtered_df)

Training dataframe size: 2263
Testing dataframe size: 961


#### Split data (data processed in the same session, not loaded)

In [ ]:
training_df, testing_df = split_data(augmented_df)

Training dataframe size: 2018
Testing dataframe size: 488


#### Training

In [ ]:
data = training_df

num_classes = len(data['label'].unique())

image_data = np.array(data['image'])
image_data = np.stack(image_data)

labels = tf.keras.utils.to_categorical(data['label'], num_classes=num_classes)

In [ ]:
# Single input Alexnet
num_classes = len(data['label'].unique())
model = AlexNet(num_classes)

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
model.fit(image_data, labels, batch_size=32, epochs=200, validation_split=0.2)

In [ ]:
# Mixed input Alexnet
num_classes = len(data['label'].unique())
model = AlexNetMixedInput(num_classes)
model.fit([image_data, category_numbers], labels, batch_size=32, epochs=500, validation_split=0.2)

#### Saving

In [ ]:
from keras.models import load_model
import tensorflowjs as tfjs

In [ ]:
model.save(os.path.join(paths['TRAINED_MODELS'], 'alexnet_v2_no_reg.h5'))

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
trained_model = load_model(os.path.join(paths['TRAINED_MODELS'], 'alexnet_v2_82.h5'))
# tf.keras.models.save_model(trained_model, os.path.join(paths['TRAINED_MODELS'], 'exports'))

In [ ]:
trained_model.optimizer.get_config()

{'name': 'SGD',
 'learning_rate': 1e-04,
 'decay': 0.0,
 'momentum': 0.9,
 'nesterov': False}

In [ ]:
tfjs.converters.save_keras_model(trained_model, os.path.join(paths['TRAINED_MODELS'], 'exports', 'tfjs'))

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Convert to TFJS
!tensorflowjs_converter --input_format keras \
  {os.path.join(paths['TRAINED_MODELS'], 'alexnet_v2_no_reg.h5')} \
  {os.path.join(paths['TRAINED_MODELS'], 'exports', 'tfjs')}

2023-10-04 19:34:57.795434: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-04 19:34:57.795484: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-04 19:34:57.795518: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-04 19:34:58.793900: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Model evaluation

In [ ]:
num_classes = len(training_df['label'].unique())

train_images = np.stack(np.array(training_df['image']))
train_categories = np.array(training_df['category'])
train_labels = tf.keras.utils.to_categorical(training_df['label'], num_classes=num_classes)

test_images = np.stack(np.array(testing_df['image']))
test_categories = np.array(testing_df['category'])
test_labels = tf.keras.utils.to_categorical(testing_df['label'], num_classes=num_classes)

learning_rates = [0.0001, 0.0005, 0.00005]
momentums = [0.9, 0.8, 0.99]
decays = [0, 0.01, 0.001]
regularizations = [0.1, 0, 0.25]

In [ ]:
loops = 0

file_path = 'workspace/classifier/training/eval_results.txt'
with open(file_path, 'a') as file:
  for lr in learning_rates:
    for moment in momentums:
      for dc in decays:
        for reg in regularizations:
          model = AlexNet(num_classes=num_classes,
                          learning_rate=lr,
                          momentum=moment,
                          decay=dc,
                          regularization=reg)
          model.fit(train_images, train_labels, batch_size=32, epochs=200, validation_split=0.2)

          predictions = model.predict(test_images)
          predicted_labels = np.argmax(predictions, axis=1)

          # Evaluate the model performance (optional)
          accuracy = np.mean(predicted_labels == np.argmax(test_labels, axis=1))

          print("#########")
          print("#########")
          print(f"Loop number: {loops}")
          print(f"{lr},{moment},{dc},{reg}: => accuracy: {accuracy}")
          print("#########")
          loops += 1

          file.write(str([lr, moment, dc, reg, accuracy]) + '\n')


In [ ]:
file_path = 'workspace/classifier/training/eval_results.txt'
with open(file_path, 'r') as file:
    for line in file:
        print(line.strip())

In [ ]:
from google.colab import runtime
runtime.unassign()

### Testing

#### Load trained model

In [ ]:
# alexnet_v2_82.h5 yields 82% on test data
model = load_model(os.path.join(paths['TRAINED_MODELS'], 'alexnet_v2_82.h5'))

#### Test model

In [ ]:
test_images = testing_image_data = np.stack(np.array(testing_df['image']))

In [ ]:
# Load your Pandas DataFrame
data = testing_df

# Preprocess your data
num_classes = len(data['label'].unique())

testing_image_data = np.stack(np.array(testing_df['image']))
labels = tf.keras.utils.to_categorical(data['label'], num_classes=num_classes)


image_test = testing_image_data
label_test = labels

# Predict using the trained model
predictions = model.predict(image_test)

predicted_labels = np.argmax(predictions, axis=1)

# Evaluate the model performance
accuracy = np.mean(predicted_labels == np.argmax(label_test, axis=1))
print(f"Accuracy on test data: {accuracy:.2f}")


16/16 [==============================] - 1s 39ms/step
Accuracy on test data: 0.81


In [ ]:
[labels[prediction] for prediction in predicted_labels]

In [ ]:
test_data_np = (np.array(testing_image_data) * 255).astype(np.uint8)

for ind in range(len(predicted_labels)):
  print(labels[predicted_labels[ind]])
  image_np = test_data_np[ind]

  # Convert the numpy array to an image
  image = Image.fromarray(image_np)

  # Display the image using matplotlib
  plt.figure()
  plt.imshow(image)
  plt.axis('off')
  plt.show()


In [ ]:
# Image input
image_input = Input(shape=(256, 256, 3), name='image_input')
conv1 = Conv2D(16, (3, 3), 1, activation='relu')(image_input)
pool1 = MaxPooling2D()(conv1)

conv2 = Conv2D(32, (3, 3), 1, activation='relu')(pool1)
pool2 = MaxPooling2D()(conv2)

conv3 = Conv2D(16, (3, 3), 1, activation='relu')(pool2)
pool3 = MaxPooling2D()(conv3)

flat = Flatten()(pool3)
dense1 = Dense(256, activation='relu')(flat)

# Category input
category_input = Input(shape=(1,), name='category_input')
dense2 = Dense(64, activation='relu')(category_input)

# Combine image and category features
combined = Concatenate()([dense1, dense2])

# Output layer for multi-class classification
output = Dense(num_classes, activation='softmax')(combined)

In [ ]:
model = Model(inputs=[image_input, category_input], outputs=output)

In [ ]:
model.compile(optimizer='adam', loss=tf.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

### Experimentation

In [ ]:
augmentation_map = [(5, 0), (6, 0), (3, 109), (13, 131), (15, 131), (2, 140), (4, 140), (12, 140),
                    (1, 140), (0, 140), (20, 140), (17, 140), (16, 90), (14, 80),
                    (18, 60), (11, 50), (19, 20), (7, 20), (9, 10), (8, 10), (10, 10)]

In [ ]:
original_df['label'].value_counts()

In [ ]:
len(augmented_df['label'].value_counts())